In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from catboost import CatBoostClassifier

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)


LOAD MODEL

In [3]:
def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH

def load_models():
    model_path = get_model_path("catboost_model")
    model = CatBoostClassifier().load_model('catboost_model', format='cbm')
    return model

In [4]:
model = load_models()


LOAD DATAFRAMES

In [5]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def load_features() -> pd.DataFrame:
    query1 = 'SELECT * FROM kokh_user_features_df'
    query2 = 'SELECT * FROM kokh_post_features_df'
    return batch_load_sql(query1), batch_load_sql(query2)

In [6]:
user_features_df, post_features_df = load_features()

FUNCTION FOR PREDICTION

In [7]:
def prediction_top_5_posts(user_features_df, post_features_df, user_id, model):

    ## Save the place for features is important for model
    places_for_features_columns = ['user_id', 'post_id', 'gender', 'age', 'country', 'city',
                                  'exp_group', 'os', 'source', 'count_actions', 'category_of_age',
                                  'cluster_feature', 'month', 'day', 'second', 'weekday', 'is_weekend',
                                  'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 
                                  'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 
                                  'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 
                                  'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 
                                  'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25',
                                  'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 
                                  'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 
                                  'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40',
                                  'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 
                                  'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50',
                                  'part_of_day', 'topic']
    
    # Create copy of dataframes and find the data of this user 
    this_user_data = user_features_df.copy().loc[user_features_df['user_id'] == user_id]
    all_post_features_df = post_features_df.copy()

    # Merge dataframes on key column
    this_user_data['key'] = 1
    all_post_features_df['key'] = 1
    result = this_user_data.merge(all_post_features_df, on='key').drop('key', axis=1)
    result = result[places_for_features_columns].set_index(['user_id', 'post_id'])
    result['prediction'] = model.predict_proba(result)[:, 1]
    top_5_posts = result.sort_values('prediction', ascending=False).head(5).index.get_level_values('post_id').tolist()
    return top_5_posts

TEST PREDICTION

In [8]:
top_5_posts_ids = prediction_top_5_posts(user_features_df, post_features_df, 160001, model)

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=20]=-0.2279909312816818 : cat_features must be integer or string, real number values and NaN values should be converted to string.

OPEN POST TABLE AND FIND TOP 5 RECOMMENDAATIONS

In [ ]:
post_text_df = pd.read_sql('SELECT * FROM public.post_text_df', con=engine)

posts = post_text_df[post_text_df['post_id'].isin(top_5_posts_ids)]

posts

In [ ]:
valid_post_ids = post_text_df['post_id'].unique().tolist()

top_5_posts_ids = [post_id for post_id in top_5_posts_ids if post_id in valid_post_ids]

top_5_posts_ids